In [1]:
from functools import total_ordering, reduce
import csv
import re
import pandas as pd
import gensim
import math
import pandas as pd
from utils import *

In [2]:
df = pd.read_csv("wiki_movie.csv")
docs = df["Plot"]
docs = docs[:1000]

In [3]:
docs = [
    "hello hello my name is giovanni",
    "fuck this shit I am going to the beach",
    "the movie last night was easily my favourite",
    "the film was a complete shit",
    "can you please go fuck yourself for fuck sake?" 
]

In [4]:
vocab, postings = create_vocab_and_postings_list(docs)

processed 0 docs


In [5]:
print(len(vocab))
print(vocab)

30
['hello', 'my', 'name', 'is', 'giovanni', 'fuck', 'this', 'shit', 'i', 'am', 'going', 'to', 'the', 'beach', 'movie', 'last', 'night', 'was', 'easily', 'favourite', 'film', 'a', 'complete', 'can', 'you', 'please', 'go', 'yourself', 'for', 'sake']


In [6]:
vocab_idf = {}
N = len(docs)

for v in vocab:
    vocab_idf[v] = math.log(N / len(postings[v]), 10)
    
vocab_idf

{'hello': 0.6989700043360187,
 'my': 0.3979400086720376,
 'name': 0.6989700043360187,
 'is': 0.6989700043360187,
 'giovanni': 0.6989700043360187,
 'fuck': 0.3979400086720376,
 'this': 0.6989700043360187,
 'shit': 0.3979400086720376,
 'i': 0.6989700043360187,
 'am': 0.6989700043360187,
 'going': 0.6989700043360187,
 'to': 0.6989700043360187,
 'the': 0.22184874961635637,
 'beach': 0.6989700043360187,
 'movie': 0.6989700043360187,
 'last': 0.6989700043360187,
 'night': 0.6989700043360187,
 'was': 0.3979400086720376,
 'easily': 0.6989700043360187,
 'favourite': 0.6989700043360187,
 'film': 0.6989700043360187,
 'a': 0.6989700043360187,
 'complete': 0.6989700043360187,
 'can': 0.6989700043360187,
 'you': 0.6989700043360187,
 'please': 0.6989700043360187,
 'go': 0.6989700043360187,
 'yourself': 0.6989700043360187,
 'for': 0.6989700043360187,
 'sake': 0.6989700043360187}

In [7]:
myquery = "fuck shit pizza"
q = preprocess_query(myquery)
print(q)

fuck shit pizza


### Cosine Similarity

In [8]:
vectorized_docs = [vectorize(i, docs, postings, vocab, vocab_idf) for i in range(len(docs))]
print(vectorized_docs[1])

[0.         0.         0.         0.         0.         0.21914897
 0.38492877 0.21914897 0.38492877 0.38492877 0.38492877 0.38492877
 0.12217401 0.38492877 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


In [9]:
vectorized_query = vectorize_query(q, vocab)
print(vectorized_query)

[0.         0.         0.         0.         0.         0.70710678
 0.         0.70710678 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


In [10]:
dot_product_scores = DotProductScore(vectorized_docs, vectorized_query)
print(dot_product_scores)

[0.         0.30992345 0.         0.20791533 0.27952855]


### CosineScore

In [11]:
scores = CosineScore(q, docs, vocab_idf, postings)
print(scores)

[0.         0.02094421 0.         0.01421214 0.01730174]


### Extract K top doc

In [27]:
K = 2

def extract_top_docs(docs, scores, k):
    docs = np.asarray(docs)
    #ind = np.argpartition(scores, -k)[-k:]
    ind = scores.argsort()[::-1][:k]
    #return ind[np.argsort(docs[ind])]
    return docs[ind]

In [29]:
extract_top_docs(docs, dot_product_scores, 2)

array(['fuck this shit I am going to the beach',
       'can you please go fuck yourself for fuck sake?'], dtype='<U46')